In [ ]:
import numpy as np
from collections import deque
from typing import List, Set, Optional, Tuple
import time

class PuzzleState:
    def __init__(self, board: np.ndarray, parent=None, move=""):
        self.board = board
        self.parent = parent
        self.move = move
        self._hash = None
        self._empty_pos = None
        self.manhattan_dist = None  # Store manhattan distance
        
    @property
    def empty_pos(self) -> Tuple[int, int]:
        if self._empty_pos is None:
            self._empty_pos = tuple(np.where(self.board == 0))
            self._empty_pos = (self._empty_pos[0][0], self._empty_pos[1][0])
        return self._empty_pos
    
    def __eq__(self, other):
        return np.array_equal(self.board, other.board)
    
    def __hash__(self):
        if self._hash is None:
            self._hash = hash(self.board.tobytes())
        return self._hash

class PuzzleSolver:
    def __init__(self, initial_state: List[List[int]]):
        self.initial_state = PuzzleState(np.array(initial_state))
        self.goal = np.arange(16).reshape(4, 4)
        self.moves_explored = 0
    
    def manhattan_distance(self, state: PuzzleState) -> int:
        """Calculate Manhattan distance for move ordering."""
        if state.manhattan_dist is None:
            distance = 0
            for i in range(4):
                for j in range(4):
                    if state.board[i][j] != 0:
                        x, y = divmod(state.board[i][j] - 1, 4)
                        distance += abs(x - i) + abs(y - j)
            state.manhattan_dist = distance
        return state.manhattan_dist
    
    def get_neighbors(self, state: PuzzleState) -> List[PuzzleState]:
        """Generate possible next states, ordered by Manhattan distance."""
        neighbors = []
        row, col = state.empty_pos
        moves = [
            (-1, 0, "down"),  # Direction names from tile perspective
            (1, 0, "up"),
            (0, -1, "right"),
            (0, 1, "left")
        ]
        
        for dx, dy, move in moves:
            new_row, new_col = row + dx, col + dy
            if 0 <= new_row < 4 and 0 <= new_col < 4:
                new_board = state.board.copy()
                new_board[row][col], new_board[new_row][new_col] = \
                    new_board[new_row][new_col], new_board[row][col]
                neighbor = PuzzleState(new_board, state, move)
                # Calculate and store manhattan distance
                dist = self.manhattan_distance(neighbor)
                neighbors.append((dist, neighbor))
        
        # Sort by manhattan distance
        neighbors.sort(key=lambda x: x[0])
        return [n[1] for n in neighbors]
    
    def is_solvable(self) -> bool:
        """Check if the puzzle is solvable."""
        # Convert board to 1D array
        flat = self.initial_state.board.flatten()
        
        # Count inversions
        inversions = 0
        for i in range(16):
            if flat[i] == 0:
                continue
            for j in range(i + 1, 16):
                if flat[j] != 0 and flat[i] > flat[j]:
                    inversions += 1
        
        # Get row of blank tile from bottom
        blank_row = 4 - self.initial_state.empty_pos[0]
        
        # Puzzle is solvable if:
        # - blank on even row from bottom + odd inversions
        # - blank on odd row from bottom + even inversions
        return (blank_row % 2 == 0 and inversions % 2 == 1) or \
               (blank_row % 2 == 1 and inversions % 2 == 0)
    
    def dfs(self, state: PuzzleState, depth_limit: int, depth: int, 
            visited: Set[int]) -> Optional[List[str]]:
        """Depth-first search with cycle detection and pruning."""
        if depth > depth_limit:
            return None
        
        if np.array_equal(state.board, self.goal):
            path = []
            current = state
            while current.parent is not None:
                path.append(current.move)
                current = current.parent
            return path[::-1]
        
        if depth == depth_limit:
            return None
        
        state_hash = hash(state)
        if state_hash in visited:
            return None
        
        visited.add(state_hash)
        self.moves_explored += 1
        
        # Get and explore neighbors
        for neighbor in self.get_neighbors(state):
            result = self.dfs(neighbor, depth_limit, depth + 1, visited)
            if result is not None:
                return result
        
        visited.remove(state_hash)
        return None
    
    def solve_ids(self, max_depth: int = 50) -> Optional[List[str]]:
        """Solve using Iterative Deepening Search."""
        if not self.is_solvable():
            return None
            
        for depth in range(max_depth):
            print(f"Searching depth {depth}...")
            visited = set()
            result = self.dfs(self.initial_state, depth, 0, visited)
            if result is not None:
                return result
        return None

def visualize_puzzle(board: np.ndarray) -> str:
    """Create ASCII visualization of the puzzle state."""
    border = "+----+----+----+----+"
    result = [border]
    
    for row in board:
        line = "|"
        for num in row:
            if num == 0:
                line += "    |"
            else:
                line += f" {num:2d} |"
        result.append(line)
        result.append(border)
    
    return "\n".join(result)

# Example usage
if __name__ == "__main__":
    # Test case (a few moves from solution)
    initial_state = [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 0],
        [13, 14, 15, 12]
    ]
    
    solver = PuzzleSolver(initial_state)
    print("Initial State:")
    print(visualize_puzzle(solver.initial_state.board))
    
    print("\nChecking solvability...")
    if not solver.is_solvable():
        print("This puzzle configuration is not solvable!")
    else:
        print("Puzzle is solvable. Solving...")
        start_time = time.time()
        solution = solver.solve_ids()
        end_time = time.time()
        
        if solution:
            print(f"\nSolution found in {end_time - start_time:.2f} seconds!")
            print("Moves:", solution)
            print(f"Number of moves: {len(solution)}")
            print(f"States explored: {solver.moves_explored}")
        else:
            print("\nNo solution found within depth limit")

Initial State:
+----+----+----+----+
|  1 |  2 |  3 |  4 |
+----+----+----+----+
|  5 |  6 |  7 |  8 |
+----+----+----+----+
|  9 | 10 | 11 |    |
+----+----+----+----+
| 13 | 14 | 15 | 12 |
+----+----+----+----+

Checking solvability...
Puzzle is solvable. Solving...
Searching depth 0...
Searching depth 1...
Searching depth 2...
Searching depth 3...
Searching depth 4...
Searching depth 5...
Searching depth 6...
Searching depth 7...
Searching depth 8...
Searching depth 9...
Searching depth 10...
Searching depth 11...
Searching depth 12...
Searching depth 13...
Searching depth 14...
Searching depth 15...
Searching depth 16...
Searching depth 17...
Searching depth 18...
Searching depth 19...
Searching depth 20...
Searching depth 21...
